In [35]:
# Importation des librairies
from my_function import ( 
    sel_classif_pixel,
    report_from_dict_to_df,
    supprimer_dossier_non_vide)
import os
from osgeo import gdal
import geopandas as gpd
import numpy as np
import pandas as pd
import sys
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, KFold, RepeatedStratifiedKFold
from sklearn.metrics import confusion_matrix, classification_report, \
    accuracy_score
sys.path.append('/home/onyxia/work/libsigma')
import read_and_write as rw
import classification as cla
import plots

In [2]:
# Création du dossier de sauvegarde temporaire et paramètres
output_dir = "/home/onyxia/work/output_classif"
os.makedirs(output_dir, exist_ok=True) # Création d'un dossier output temporaire

sample_path = "/home/onyxia/work/Depot_Git/results/data/sample/Sample_BD_foret_T31TCJ.shp"
path_sample_px = os.path.join(output_dir,"sample_classif_px.shp")

sample_rasterized = os.path.join(output_dir,"rasterized_sample.tif")
path_image_3b = "/home/onyxia/work/Depot_Git/results/data/img_pretraitees/Serie_temp_S2_3_bands.tif"
path_image_10b = "/home/onyxia/work/Depot_Git/results/data/img_pretraitees/Serie_temp_S2_10_bands.tif"
path_image_allbands = "/home/onyxia/work/Depot_Git/results/data/img_pretraitees/Serie_temp_S2_allbands.tif"

path_sample_px_centroid = os.path.join(output_dir,"sample_px_centroid.shp")

In [3]:
# Save d'un vecteur echantillons avec que les données pour classif pixel
sample = gpd.read_file(sample_path)
sample_px = sel_classif_pixel(sample[['Code',"geometry"]])
sample_px.to_file(path_sample_px)

In [4]:
# cmd rasterisation paramètres
in_vector = os.path.join(output_dir,'sample_classif_px.shp')
ref_image = '/home/onyxia/work/Depot_Git/results/data/img_pretraitees/Serie_temp_S2_allbands.tif'
ref_image_open = rw.open_image(ref_image)
out_image = os.path.join(output_dir, "rasterized_sample.tif")
field_name = 'Code'  # field containing the numeric label of the classes
sptial_resolution = rw.get_pixel_size(ref_image_open)[0]
xmin,ymax = rw.get_origin_coordinates(ref_image_open)
y,x = rw.get_image_dimension(ref_image_open)[0:2]
xmax,ymin = xmin+x*10,ymax-y*10

# define command pattern to fill with paremeters
cmd_pattern = ("gdal_rasterize -a {field_name} "
               "-tr {sptial_resolution} {sptial_resolution} "
               "-te {xmin} {ymin} {xmax} {ymax} -ot Byte -of GTiff "
               "{in_vector} {out_image}")

# fill the string with the parameter thanks to format function
cmd = cmd_pattern.format(in_vector=in_vector, xmin=xmin, ymin=ymin, xmax=xmax,
                         ymax=ymax, out_image=out_image, field_name=field_name,
                         sptial_resolution=sptial_resolution)

# execute the command in the terminal
os.system(cmd)

/opt/conda/lib/python3.12/site-packages/osgeo/gdal.py:311: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


0...10...20...30...40...50...60...70...80...90...100 - done.


0

In [5]:
# Extraction des échantillons à partir d'un fichier de points pour tenir compte de l'appartenance aux polygones

sample_px['geom_points'] = sample_px.centroid                  # Calcul des centroides
sample_px_centroid = sample_px[['Code','geom_points']]
sample_px_centroid.to_file(path_sample_px_centroid)      # Save des centroides dans le dossier output_classif

# Récupération des coordonnées X
list_row, list_col = rw.get_row_col_from_file (path_sample_px_centroid,path_image_allbands)     # Coordonnées images des points du fichiers points
image = rw.load_img_as_array(path_image_allbands)
X = image[(list_row, list_col)]

# Récupération des coordonnées Y
gdf = gpd.read_file(path_sample_px_centroid)             
Y = gdf.loc[:, "Code"].values
Y = np.atleast_2d(Y).T

# Séparation des données avec un jeu train et test
# X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.7)

/tmp/ipykernel_49822/2955711469.py:5: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  sample_px_centroid.to_file(path_sample_px_centroid)      # Save des centroides dans le dossier output_classif


In [9]:
Y

array([[12],
       [12],
       [12],
       ...,
       [12],
       [12],
       [12]], shape=(2679, 1), dtype=int32)

In [8]:
# Application validation statifiée avec appartenance à polygones répétées 30 fois

nb_iter = 30
nb_folds = 5

# outputs
suffix = '_CV{}fold_stratifed_x{}times'.format(nb_folds, nb_iter)
out_classif = os.path.join(output_dir, 'ma_classif{}.tif'.format(suffix))
out_matrix = os.path.join(output_dir, 'ma_matrice{}.png'.format(suffix))
out_qualite = os.path.join(output_dir, 'mes_qualites{}.png'.format(suffix))

list_cm = []
list_accuracy = []
list_report = []

# Iter on stratified K fold
kf = RepeatedStratifiedKFold(n_splits=nb_folds, n_repeats=nb_iter)
for train, test in kf.split(X, Y):
    X_train, X_test = X[train], X[test]
    Y_train, Y_test = Y[train], Y[test]

    # Modèle Random Forest Classifier
    rfc = RandomForestClassifier(
        max_depth = 50,
        oob_score = True,
        max_samples = 0.75,
        class_weight = 'balanced',
        n_jobs = -1
    )
    rfc.fit(X_train, Y_train)

    Y_predict = rfc.predict(X_test)

    # compute quality
    list_cm.append(confusion_matrix(Y_test, Y_predict))
    list_accuracy.append(accuracy_score(Y_test, Y_predict))
    report = classification_report(Y_test, Y_predict, labels=np.unique(Y_predict), output_dict=True)

    # store them
    list_report.append(report_from_dict_to_df(report))

/opt/conda/lib/python3.12/site-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


{'11': {'precision': 0.7, 'recall': 0.2692307692307692, 'f1-score': 0.3888888888888889, 'support': 26.0}, '12': {'precision': 0.8741865509761388, 'recall': 0.9264367816091954, 'f1-score': 0.8995535714285714, 'support': 435.0}, '13': {'precision': 1.0, 'recall': 0.08333333333333333, 'f1-score': 0.15384615384615385, 'support': 12.0}, '14': {'precision': 0.7142857142857143, 'recall': 0.29411764705882354, 'f1-score': 0.4166666666666667, 'support': 34.0}, '23': {'precision': 0.02857142857142857, 'recall': 0.3333333333333333, 'f1-score': 0.05263157894736842, 'support': 3.0}, '24': {'precision': 0.7142857142857143, 'recall': 0.5, 'f1-score': 0.5882352941176471, 'support': 20.0}, '25': {'precision': 1.0, 'recall': 0.25, 'f1-score': 0.4, 'support': 4.0}, 'micro avg': {'precision': 0.8078358208955224, 'recall': 0.8108614232209738, 'f1-score': 0.8093457943925234, 'support': 534.0}, 'macro avg': {'precision': 0.7187613440169994, 'recall': 0.37949312350935066, 'f1-score': 0.41426030769932803, 'supp

/opt/conda/lib/python3.12/site-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


{'11': {'precision': 0.75, 'recall': 0.11538461538461539, 'f1-score': 0.2, 'support': 26.0}, '12': {'precision': 0.8791946308724832, 'recall': 0.903448275862069, 'f1-score': 0.891156462585034, 'support': 435.0}, '13': {'precision': 1.0, 'recall': 0.09090909090909091, 'f1-score': 0.16666666666666666, 'support': 11.0}, '14': {'precision': 0.6, 'recall': 0.2647058823529412, 'f1-score': 0.3673469387755102, 'support': 34.0}, '23': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 3.0}, '24': {'precision': 0.64, 'recall': 0.8, 'f1-score': 0.7111111111111111, 'support': 20.0}, '25': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 5.0}, 'micro avg': {'precision': 0.7873134328358209, 'recall': 0.7902621722846442, 'f1-score': 0.788785046728972, 'support': 534.0}, 'macro avg': {'precision': 0.5527420901246405, 'recall': 0.3106354092155309, 'f1-score': 0.33375445416261745, 'support': 534.0}, 'weighted avg': {'precision': 0.8354862629766483, 'recall': 0.7902621722846442, '

/opt/conda/lib/python3.12/site-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


...

{'11': {'precision': 0.5, 'recall': 0.08, 'f1-score': 0.13793103448275862, 'support': 25.0}, '12': {'precision': 0.8903225806451613, 'recall': 0.9539170506912442, 'f1-score': 0.9210233592880979, 'support': 434.0}, '14': {'precision': 0.8333333333333334, 'recall': 0.42857142857142855, 'f1-score': 0.5660377358490566, 'support': 35.0}, '23': {'precision': 0.045454545454545456, 'recall': 0.25, 'f1-score': 0.07692307692307693, 'support': 4.0}, '24': {'precision': 0.5833333333333334, 'recall': 0.7368421052631579, 'f1-score': 0.6511627906976745, 'support': 19.0}, '25': {'precision': 0.6666666666666666, 'recall': 0.4, 'f1-score': 0.5, 'support': 5.0}, 'micro avg': {'precision': 0.835820895522388, 'recall': 0.8582375478927203, 'f1-score': 0.8468809073724007, 'support': 522.0}, 'macro avg': {'precision': 0.5865184099055066, 'recall': 0.4748884307543051, 'f1-score': 0.4755129995401108, 'support': 522.0}, 'weighted avg': {'precision': 0.8480175316382214, 'recall': 0.8582375478927203, 'f1-score': 0

/opt/conda/lib/python3.12/site-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


{'11': {'precision': 0.6666666666666666, 'recall': 0.32, 'f1-score': 0.43243243243243246, 'support': 25.0}, '12': {'precision': 0.8891352549889135, 'recall': 0.923963133640553, 'f1-score': 0.9062146892655367, 'support': 434.0}, '14': {'precision': 0.7272727272727273, 'recall': 0.22857142857142856, 'f1-score': 0.34782608695652173, 'support': 35.0}, '23': {'precision': 0.02631578947368421, 'recall': 0.25, 'f1-score': 0.047619047619047616, 'support': 4.0}, '24': {'precision': 0.6086956521739131, 'recall': 0.7368421052631579, 'f1-score': 0.6666666666666666, 'support': 19.0}, '25': {'precision': 1.0, 'recall': 0.2, 'f1-score': 0.3333333333333333, 'support': 5.0}, 'micro avg': {'precision': 0.8078358208955224, 'recall': 0.8295019157088123, 'f1-score': 0.8185255198487713, 'support': 522.0}, 'macro avg': {'precision': 0.6530143484293175, 'recall': 0.44322944457918995, 'f1-score': 0.4556820427122565, 'support': 522.0}, 'weighted avg': {'precision': 0.8518704853172409, 'recall': 0.82950191570881

/opt/conda/lib/python3.12/site-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


{'11': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 26.0}, '12': {'precision': 0.8620689655172413, 'recall': 0.9216589861751152, 'f1-score': 0.89086859688196, 'support': 434.0}, '14': {'precision': 0.46153846153846156, 'recall': 0.17647058823529413, 'f1-score': 0.2553191489361702, 'support': 34.0}, '23': {'precision': 0.02857142857142857, 'recall': 0.25, 'f1-score': 0.05128205128205128, 'support': 4.0}, '24': {'precision': 0.6842105263157895, 'recall': 0.6842105263157895, 'f1-score': 0.6842105263157895, 'support': 19.0}, '25': {'precision': 1.0, 'recall': 0.25, 'f1-score': 0.4, 'support': 4.0}, 'micro avg': {'precision': 0.7869158878504673, 'recall': 0.8080614203454894, 'f1-score': 0.7973484848484849, 'support': 521.0}, 'macro avg': {'precision': 0.5060648969904867, 'recall': 0.38039001678769985, 'f1-score': 0.38028005390266184, 'support': 521.0}, 'weighted avg': {'precision': 0.7810835401940041, 'recall': 0.8080614203454894, 'f1-score': 0.7871841656731836, 'support': 

In [ ]:
# compute mean of cm
array_cm = np.array(list_cm)
mean_cm = array_cm.mean(axis=0)

# compute mean and std of overall accuracy
array_accuracy = np.array(list_accuracy)
mean_accuracy = array_accuracy.mean()
std_accuracy = array_accuracy.std()

# compute mean and std of classification report
array_report = np.array(list_report)
mean_report = array_report.mean(axis=0)
std_report = array_report.std(axis=0)
a_report = list_report[0]
mean_df_report = pd.DataFrame(mean_report, index=a_report.index,
                              columns=a_report.columns)
std_df_report = pd.DataFrame(std_report, index=a_report.index,
                             columns=a_report.columns)

# Display confusion matrix
plots.plot_cm(mean_cm, np.unique(Y_predict))
plt.savefig(out_matrix, bbox_inches='tight')

# Display class metrics
fig, ax = plt.subplots(figsize=(10, 7))
ax = mean_df_report.T.plot.bar(ax=ax, yerr=std_df_report.T, zorder=2)
ax.set_ylim(0.5, 1)
_ = ax.text(1.5, 0.95, 'OA : {:.2f} +- {:.2f}'.format(mean_accuracy,
                                                      std_accuracy),
            fontsize=14)
ax.set_title('Class quality estimation')

# custom : cuteness
# background color
ax.set_facecolor('ivory')
# labels
x_label = ax.get_xlabel()
ax.set_xlabel(x_label, fontdict={'fontname': 'Sawasdee'}, fontsize=14)
y_label = ax.get_ylabel()
ax.set_ylabel(y_label, fontdict={'fontname': 'Sawasdee'}, fontsize=14)
# borders
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
ax.spines["bottom"].set_visible(False)
ax.spines["left"].set_visible(False)
ax.tick_params(axis='x', colors='darkslategrey', labelsize=14)
ax.tick_params(axis='y', colors='darkslategrey', labelsize=14)
# grid
ax.minorticks_on()
ax.yaxis.grid(which='major', color='darkgoldenrod', linestyle='--',
              linewidth=0.5, zorder=1)
ax.yaxis.grid(which='minor', color='darkgoldenrod', linestyle='-.',
              linewidth=0.3, zorder=1)
plt.savefig(out_qualite, bbox_inches='tight')

In [11]:
X_img, _, t_img = cla.get_samples_from_roi(path_image_allbands, path_image_allbands)

In [12]:
Y_predict = rfc.predict(X_img)

In [13]:
image_filename = path_image_allbands

# Get image dimension
ds = rw.open_image(image_filename)
nb_row, nb_col, _ = rw.get_image_dimension(ds)

#initialization of the array
img = np.zeros((nb_row, nb_col, 1), dtype='uint8')
#np.Y_predict

img[t_img[0], t_img[1], 0] = Y_predict

In [27]:
img

array([[[0],
        [0],
        [0],
        ...,
        [0],
        [0],
        [0]],

       [[0],
        [0],
        [0],
        ...,
        [0],
        [0],
        [0]],

       [[0],
        [0],
        [0],
        ...,
        [0],
        [0],
        [0]],

       ...,

       [[0],
        [0],
        [0],
        ...,
        [0],
        [0],
        [0]],

       [[0],
        [0],
        [0],
        ...,
        [0],
        [0],
        [0]],

       [[0],
        [0],
        [0],
        ...,
        [0],
        [0],
        [0]]], shape=(7381, 10863, 1), dtype=uint8)

In [37]:
image_filename =path_image_allbands

out_filename = '/home/onyxia/work/Depot_Git/results/data/classif/carte_essences_echelle_pixel.tif'

# get reference dataset
ds = rw.open_image(image_filename)

# just replace the number of band (1 instead of 3)
rw.write_image(out_filename, img, data_set=ds, gdal_dtype=gdal.GDT_Byte,
                transform=None, projection=None, driver_name=None,
                nb_col=None, nb_ligne=None, nb_band=1)

In [38]:
## Nettoyage des dossiers
supprimer_dossier_non_vide(output_dir)